<h1 align="center">SFM WARC Analysis</h1> 

#### This notebook is for processing SFM WARCs 



<h3 align="center">Setup</h3> 

### Set data directory

In [1]:
sfm_warc_data_dir = '/ars-data/'

### Define Helper functions

In [2]:
import os
import json
import gzip
import warc
import fnmatch
from StringIO import StringIO
import re

def is_json_record(record):
    return record['Content-Type'] == 'application/http;msgtype=response'

def extract_json_data_from_record(record):
    payload = record.payload.read()
    splits = re.split("\r\n\r\n", payload)
    data = splits[-1]
    uncompressed_data = json.loads(gzip.GzipFile(fileobj=StringIO(data)).read())
    return uncompressed_data
    
def warc_processor(warc_data_dir, extract_func):
    extracted_data = []
    for dirpath, dirs, files in os.walk(sfm_warc_data_dir):
        for filename in fnmatch.filter(files, '*.warc.gz'):
            f = warc.WARCFile(fileobj=gzip.open(os.path.join(dirpath, filename)))
            file_extracted_data = filter(lambda x: x is not None, map(extract_func, f))
            extracted_data += file_extracted_data
    return extracted_data        



<h3 align="center">Explore</h3> 


In [3]:
def load_json_record(record):
    if not is_json_record(record):
        return None
    data = extract_json_data_from_record(record)
    return data

#Load JSON data from SFM WARCs
json_metadata = warc_processor(sfm_warc_data_dir, load_json_record)

In [4]:
#print example record
print(json.dumps(json_metadata[1]))

[{"contributors": null, "truncated": false, "text": "It's time to get to work to make sure we never hear the words \"President Trump.\" https://t.co/iCgAxZHFv7 https://t.co/RMZYUHmfxE", "is_quote_status": false, "in_reply_to_status_id": null, "id": 728002039083601920, "favorite_count": 227, "source": "<a href=\"https://about.twitter.com/products/tweetdeck\" rel=\"nofollow\">TweetDeck</a>", "retweeted": false, "coordinates": null, "entities": {"symbols": [], "user_mentions": [], "hashtags": [], "urls": [{"url": "https://t.co/iCgAxZHFv7", "indices": [81, 104], "expanded_url": "http://bit.ly/1ZcOhe7", "display_url": "bit.ly/1ZcOhe7"}], "media": [{"expanded_url": "http://twitter.com/TheDemocrats/status/728002039083601920/photo/1", "display_url": "pic.twitter.com/RMZYUHmfxE", "url": "https://t.co/RMZYUHmfxE", "media_url_https": "https://pbs.twimg.com/media/Chph0yMXEAAjo-c.jpg", "id_str": "728001849228464128", "sizes": {"small": {"h": 170, "resize": "fit", "w": 340}, "large": {"h": 512, "res